In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [3]:
df = pd.read_csv('augmented-ecom-churn-data.csv')
df.head()

,Unnamed: 0,visitorid,ses_rec,ses_rec_avg,ses_rec_sd,user_rec,int_n_r,tran_n_r,rev_sum_r,major_spend_r,int_itm_n_avg,ses_mo_avg,ses_mo_sd,ses_ho_avg,ses_ho_sd,ses_wknd_r,ses_len_avg,time_to_int,target_class
0,0,64931,2.0,2.000000,3.807887,98.0,4.000000,0.121951,4423.9023,0.024390,2.487805,5.853659,1.062096,15.292683,4.290943,0.390244,7.451220,1.862805,0
1,1,72373,13.0,8.000000,0.000000,21.0,3.500000,0.500000,9300.0000,0.500000,1.500000,7.500000,0.707107,18.500000,2.121320,0.500000,17.175000,4.907143,0
2,2,150651,8.0,9.444445,11.596083,97.0,5.700000,0.800000,14976.0000,0.100000,3.300000,5.900000,1.100505,19.900000,1.523884,0.200000,7.818333,1.371638,0
3,3,500130,12.0,6.000000,6.164414,36.0,5.600000,0.800000,17160.0000,0.200000,4.400000,7.400000,0.547723,12.400000,8.848729,0.600000,3.943333,0.704167,0
4,4,543762,1.0,1.250000,2.046338,15.0,3.888889,0.222222,17733.3340,0.111111,2.000000,8.000000,0.000000,17.888890,3.179797,0.333333,8.664815,2.228095,0


In [4]:
df.shape

(108810, 19)

In [ ]:
columns_to_remove = df.columns[df.columns.str.startswith('int_cat') & df.columns.str.endswith('_n')]

df= df.drop(columns=columns_to_remove)


In [ ]:
df.shape

(49358, 26)

In [5]:
df.columns

Index(['Unnamed: 0', 'visitorid', 'ses_rec', 'ses_rec_avg', 'ses_rec_sd',
       'user_rec', 'int_n_r', 'tran_n_r', 'rev_sum_r', 'major_spend_r',
       'int_itm_n_avg', 'ses_mo_avg', 'ses_mo_sd', 'ses_ho_avg', 'ses_ho_sd',
       'ses_wknd_r', 'ses_len_avg', 'time_to_int', 'target_class'],
      dtype='object')

In [8]:
#X = df[['int_n_r','ses_len_avg','time_to_tran','tran_n_r','user_rec']]
X = df[['int_n_r','ses_len_avg','tran_n_r','user_rec']]
Y = df.target_class
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [10]:
max_features = None
maxlen = 100
embedding_size = None
# Convolution
kernel_size = 5
filters = 64
pool_size = 4
# LSTM
lstm_output_size = 70
# Training
batch_size = 30
epochs = 5

In [11]:
model = Sequential()
model.add(Conv1D(filters=filters,
                 kernel_size=kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, 1)))  # Input shape for Conv1D layer
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 96, 64)            384       
                                                                 
 max_pooling1d (MaxPooling1  (None, 24, 64)            0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 70)                37800     
                                                                 
 dense (Dense)               (None, 1)                 71        
                                                                 
Total params: 38255 (149.43 KB)
Trainable params: 38255 (149.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
X_train_array = X_train.values  # Convert DataFrame to NumPy array
maxlen = 100  # Desired sequence length
X_train_padded = np.zeros((len(X_train_array), maxlen, 1))  # Create an array for padded sequences

# Copy the values of X_train_array to X_train_padded with padding/truncation
for i, seq in enumerate(X_train_array):
    seq_len = len(seq)
    if seq_len >= maxlen:
        X_train_padded[i, :, :] = seq[:maxlen].reshape(-1, 1)  # Truncate if sequence is longer than maxlen
    else:
        X_train_padded[i, :seq_len, :] = seq.reshape(-1, 1)

In [13]:
model.fit(X_train_padded, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/5
2467/2467 [==============================] - 56s 22ms/step - loss: 0.6022 - accuracy: 0.6619 - val_loss: 0.5712 - val_accuracy: 0.7066
Epoch 2/5
2467/2467 [==============================] - 52s 21ms/step - loss: 0.5112 - accuracy: 0.7424 - val_loss: 0.4774 - val_accuracy: 0.7782
Epoch 3/5
2467/2467 [==============================] - 53s 21ms/step - loss: 0.4666 - accuracy: 0.7743 - val_loss: 0.4708 - val_accuracy: 0.7955
Epoch 4/5
2467/2467 [==============================] - 54s 22ms/step - loss: 0.3860 - accuracy: 0.8212 - val_loss: 0.3531 - val_accuracy: 0.8501
Epoch 5/5
2467/2467 [==============================] - 54s 22ms/step - loss: 0.3739 - accuracy: 0.8242 - val_loss: 0.3915 - val_accuracy: 0.8118


In [14]:
X_test_array = X_test.values  # Convert DataFrame to NumPy array
maxlen = 100  # Desired sequence length
X_test_padded = np.zeros((len(X_test_array), maxlen, 1))  # Create an array for padded sequences

# Copy the values of X_test_array to X_test_padded with padding/truncation
for i, seq in enumerate(X_test_array):
    seq_len = len(seq)
    if seq_len >= maxlen:
        X_test_padded[i, :, :] = seq[:maxlen].reshape(-1, 1)  # Truncate if sequence is longer than maxlen
    else:
        X_test_padded[i, :seq_len, :] = seq.reshape(-1, 1)

In [15]:
accr=model.evaluate(X_test_padded, Y_test)

511/511 [==============================] - 4s 8ms/step - loss: 0.3872 - accuracy: 0.8117


In [16]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.387
  Accuracy: 0.812


In [ ]:
#Without Padding
maxlen = 5  # Assuming your input sequences have a maximum length of 5

# Other parameters remain the same
kernel_size = 5
filters = 64
pool_size = 2
lstm_output_size = 70
batch_size = 30
epochs = 5


In [ ]:
#Without Padding
model = Sequential()
model.add(Conv1D(filters=filters,
                 kernel_size=kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, 1)))  # Input shape for Conv1D layer
# Removed MaxPooling1D layer
model.add(LSTM(lstm_output_size))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 1, 64)             384       
                                                                 
 lstm_4 (LSTM)               (None, 70)                37800     
                                                                 
 dense_4 (Dense)             (None, 1)                 71        
                                                                 
Total params: 38255 (149.43 KB)
Trainable params: 38255 (149.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Without Padding
model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/5
1119/1119 [==============================] - 9s 5ms/step - loss: 0.3479 - accuracy: 0.8854 - val_loss: 0.3465 - val_accuracy: 0.8844
Epoch 2/5
1119/1119 [==============================] - 5s 5ms/step - loss: 0.3444 - accuracy: 0.8858 - val_loss: 0.3470 - val_accuracy: 0.8845
Epoch 3/5
1119/1119 [==============================] - 5s 4ms/step - loss: 0.3438 - accuracy: 0.8859 - val_loss: 0.3473 - val_accuracy: 0.8845
Epoch 4/5
1119/1119 [==============================] - 5s 4ms/step - loss: 0.3434 - accuracy: 0.8859 - val_loss: 0.3468 - val_accuracy: 0.8845
Epoch 5/5
1119/1119 [==============================] - 5s 5ms/step - loss: 0.3435 - accuracy: 0.8859 - val_loss: 0.3487 - val_accuracy: 0.8845


In [ ]:
#Without Padding
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

232/232 [==============================] - 1s 2ms/step - loss: 0.3467 - accuracy: 0.8855
Test Loss: 0.3466593325138092, Test Accuracy: 0.8854672908782959
